# 对抗攻击

## 1. 实验介绍

1. **攻击类型**：采用非定向对抗攻击，即不指定将图像误分类为特定目标，只要求模型输出错误分类结果，将考虑L₂范数或L∞范数限制下的攻击，力求以很小的扰动降低模型整体准确率。

2. **数据集**：使用CIFAR-10数据集作为测试平台。

3. **白盒模型**：选择一个在CIFAR-10上预训练的ResNet-20卷积神经网络作为白盒攻击的目标模型。ResNet-20是针对CIFAR-10训练的残差网络，共20层，包括多个残差块，能达到约92%的测试准确率。本实验从开源仓库加载该预训练模型权重，作为攻击对象。

4. **白盒攻击方法**：采用DeepFool算法对上述白盒模型进行对抗攻击。

5. **代理模型训练**：构建一个轻量级的代理（替身）模型，例如一个小型的卷积神经网络（较浅的CNN结构），在CIFAR-10训练集上进行训练，使其学习近似目标模型的判别边界。代理模型的性能虽然可能低于ResNet-20，但在主要任务上具有相似的决策趋势，可用于黑盒攻击。

6. **黑盒攻击实施**：对代理模型应用一种简单高效的攻击算法FGSM，得到扰动后的图像，然后将这些图像输入目标模型，观察目标模型的分类准确率下降情况。

7. **结果对比与分析**：通过表格汇总原始模型的准确率、白盒攻击后准确率、黑盒攻击后准确率，以及各类别上的精度变化，对比白盒与黑盒攻击效果的差异。分析DeepFool（白盒）攻击与迁移攻击（黑盒）在降低模型准确率方面的威力和局限，并给出相应的可视化结果和结论。


## 2. 数据集加载与预训练模型准备

首先，加载CIFAR-10数据集，并准备好预训练的ResNet-20模型参数。CIFAR-10自带训练集和测试集，所以使用测试集评估模型准确率，以及作为攻击的目标输入。

In [2]:
import torch
import torchvision
import torchvision.transforms as transforms

# 数据变换：将图像转换为Tensor，并进行标准化（CIFAR-10的像素值范围0-1）
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2470, 0.2435, 0.2616))  # 常用的CIFAR-10均值和标准差
])

# 加载CIFAR-10训练集和测试集
trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
testset  = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=128, shuffle=True)
testloader  = torch.utils.data.DataLoader(testset, batch_size=100, shuffle=False)

print(f"训练集样本数: {len(trainset)}, 测试集样本数: {len(testset)}")

训练集样本数: 50000, 测试集样本数: 10000


接下来使用开源项目提供的工具函数来加载ResNet-20的预训练模型参数。

In [3]:
from pytorchcv.model_provider import get_model as ptcv_get_model

# 加载ResNet-20预训练模型（CIFAR-10）
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = ptcv_get_model("resnet20_cifar10", pretrained=True)  # ResNet-20 for CIFAR-10
model = model.to(device)
model.eval()  # 推理模式

# 打印模型结构
print(model)

CIFARResNet(
  (features): Sequential(
    (init_block): ConvBlock(
      (conv): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (activ): ReLU(inplace=True)
    )
    (stage1): Sequential(
      (unit1): ResUnit(
        (body): ResBlock(
          (conv1): ConvBlock(
            (conv): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
            (bn): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (activ): ReLU(inplace=True)
          )
          (conv2): ConvBlock(
            (conv): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
            (bn): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          )
        )
        (activ): ReLU(inplace=True)
      )
      (unit2): ResUnit(
        (body): ResBlock(
          (co

将模型切换到eval模式（`model.eval()`），评估预训练模型的整体准确率和逐类准确率：

In [4]:
# 评估模型在测试集上的准确率（无攻击情况下）
correct = 0
total = 0
class_correct = [0]*10
class_total = [0]*10

with torch.no_grad():
    for images, labels in testloader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        # 统计整体准确率
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        # 统计每类的准确率
        for i in range(labels.size(0)):
            label = labels[i]
            if predicted[i] == label:
                class_correct[label] += 1
            class_total[label] += 1

orig_acc = 100 * correct / total
print(f"模型原始整体准确率: {orig_acc:.2f}%")

# 打印每个类别的准确率
classes = testset.classes  # CIFAR-10类别名称
for i, classname in enumerate(classes):
    acc = 100 * class_correct[i] / class_total[i]
    print(f"类别 {classname:10s} 准确率: {acc:.2f}%")

模型原始整体准确率: 93.89%
类别 airplane   准确率: 95.20%
类别 automobile 准确率: 97.30%
类别 bird       准确率: 92.10%
类别 cat        准确率: 85.50%
类别 deer       准确率: 95.70%
类别 dog        准确率: 88.30%
类别 frog       准确率: 96.50%
类别 horse      准确率: 95.90%
类别 ship       准确率: 95.70%
类别 truck      准确率: 96.70%


## 3. 白盒攻击

现在在白盒设定下攻击ResNet-20模型。

选择DeepFool算法来生成对抗样本。


In [ ]:
def deepfool_attack(image, model, num_classes=10, overshoot=0.02, max_iter=50):
    """
    对单张图像应用DeepFool攻击，返回生成的对抗样本。
    参数:
        image: 原始图像tensor，形状(1,3,32,32)，requires_grad=True
        model: 被攻击的模型
        num_classes: 分类数（CIFAR-10为10）
        overshoot: 超调系数，用于加速收敛
        max_iter: 最大迭代次数
    """
    image = image.clone().detach().to(device)
    image.requires_grad = True
    fx = model(image)  # 初始模型输出
    _, orig_label = torch.max(fx, 1)
    orig_label = orig_label.item()
    
    pert_image = image.clone().detach()
    r_tot = torch.zeros_like(image).to(device)  # 累计扰动
    
    loop_i = 0
    while loop_i < max_iter:
        pert_image.requires_grad = True
        outputs = model(pert_image)
        orig_score = outputs[0, orig_label]
        # 计算原始类别的梯度
        model.zero_grad()
        orig_score.backward(retain_graph=True)
        grad_orig = pert_image.grad.data.clone()
        
        # 初始化最小扰动
        min_ratio = float('inf')
        pert = None
        new_label = orig_label
        
        # 针对每个非原始类别计算扰动距离
        for k in range(num_classes):
            if k == orig_label:
                continue
            # 计算当前类别k的梯度
            model.zero_grad()
            score_k = outputs[0, k]
            score_k.backward(retain_graph=True)
            grad_k = pert_image.grad.data.clone()
            
            # 计算针对判别面f_k - f_orig = 0的扰动大小
            w_k = grad_k - grad_orig  # 判别面法向量
            f_k = outputs[0, k] - orig_score  # 当前两类别 score 差
            # 避免除零
            if torch.norm(w_k) == 0:
                continue
            # 计算需要的扰动幅度（L2范数距离）
            ratio = abs(f_k.item()) / torch.norm(w_k).item()
            if ratio < min_ratio:
                min_ratio = ratio
                pert = w_k  # 对应最小扰动的方向
                new_label = k
        # 计算并累加扰动 r_i
        # 添加一个overshoot系数，以确保跨过边界
        r_i = (min_ratio + 1e-4) * pert / torch.norm(pert)  
        r_tot = r_tot + r_i
        # 产生对抗样本并剪裁到有效像素范围 [0,1]
        pert_image = image + (1+overshoot) * r_tot
        pert_image = pert_image.detach()
        pert_image = torch.clamp(pert_image, 0.0, 1.0)
        
        # 检查是否成功欺骗
        outputs = model(pert_image)
        _, pred_label = torch.max(outputs, 1)
        if pred_label.item() != orig_label:
            # 攻击成功，跳出
            pert_image.requires_grad = False
            break
        loop_i += 1
    
    return pert_image.detach()

# 测试DeepFool函数在一张图像上的效果
data_iter = iter(testloader)
images, labels = next(data_iter)
img0 = images[0:1].to(device)
adv_img0 = deepfool_attack(img0, model)
orig_pred = model(img0).argmax(1).item()
adv_pred = model(adv_img0).argmax(1).item()
print(f"原始标签: {labels[0].item()}, 原始预测: {orig_pred}, 攻击后预测: {adv_pred}")

原始标签: 3, 原始预测: 3, 攻击后预测: 5


对单个输入图像计算DeepFool扰动：针对每个非原始类别计算需要跨越判别边界的最小步长`ratio`，选择最小的那个作为当前迭代的扰动方向和幅度，将图像推向最近的错误分类边界。叠加每次的扰动`r_tot`，并在每次迭代后检查模型预测是否发生变化。如果发生改变则攻击成功。

第一张图像真实类别为3（猫），模型最初也预测为3，但加入DeepFool扰动后模型预测变为5（狗）。

接下来对整个测试集施加DeepFool攻击，并统计攻击后的模型准确率以及每个类别上的准确率：

In [6]:
# 对测试集进行DeepFool攻击，统计成功率
attack_correct = 0
attack_total = 0
class_correct_attack = [0]*10
class_total_attack = [0]*10

for i, (images, labels) in enumerate(testloader):
    batch_size = images.size(0)
    for j in range(batch_size):
        x = images[j:j+1].to(device)
        y = labels[j].to(device)
        # 计算对抗样本
        x_adv = deepfool_attack(x, model)
        # 用目标模型预测对抗样本
        output_adv = model(x_adv)
        _, pred = torch.max(output_adv, 1)
        attack_total += 1
        if pred.item() == y.item():
            attack_correct += 1
            class_correct_attack[y.item()] += 1
        class_total_attack[y.item()] += 1

print(f"攻击后整体准确率: {100 * attack_correct / attack_total:.2f}%  (基于{attack_total}张测试图像)")
for i, classname in enumerate(classes):
    if class_total_attack[i] > 0:
        acc = 100 * class_correct_attack[i] / class_total_attack[i]
    else:
        acc = 0.0
    print(f"攻击后类别 {classname:10s} 准确率: {acc:.2f}%")

攻击后整体准确率: 9.44%  (基于10000张测试图像)
攻击后类别 airplane   准确率: 24.60%
攻击后类别 automobile 准确率: 10.70%
攻击后类别 bird       准确率: 16.40%
攻击后类别 cat        准确率: 6.60%
攻击后类别 deer       准确率: 3.60%
攻击后类别 dog        准确率: 3.60%
攻击后类别 frog       准确率: 4.80%
攻击后类别 horse      准确率: 13.60%
攻击后类别 ship       准确率: 3.60%
攻击后类别 truck      准确率: 6.90%


## 4. 构建代理模型

模拟黑盒攻击场景：攻击者只能查询模型输出。常见方法是训练一个代理模型来近似目标模型，然后对代理模型执行攻击，再将产生的对抗样本用于目标模型，出现**迁移攻击**效果。迁移攻击利用了不同模型对相似对抗扰动可能存在的一致脆弱性。

构建一个简化的卷积神经网络作为代理模型。为了让代理模型近似目标模型，可以使用目标模型的训练数据和标签来训练代理模型，直接用CIFAR-10训练集及其真实标签来训练代理模型，使其达到较高准确率，从而提高它与目标模型决策的一致性。

定义一个小型CNN网络，包括两个卷积层和两个全连接层：

In [11]:
import torch.nn as nn
import torch.optim as optim

class SmallCNN(nn.Module):
    def __init__(self):
        super(SmallCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)
        self.pool  = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.fc1   = nn.Linear(64*8*8, 128)
        self.fc2   = nn.Linear(128, 10)
    def forward(self, x):
        x = self.pool(nn.functional.relu(self.conv1(x)))
        x = self.pool(nn.functional.relu(self.conv2(x)))
        x = x.view(-1, 64*8*8)
        x = nn.functional.relu(self.fc1(x))
        x = self.fc2(x)
        return x

proxy_model = SmallCNN().to(device)
print(proxy_model)

SmallCNN(
  (conv1): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (fc1): Linear(in_features=4096, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=10, bias=True)
)


`SmallCNN`结构相对简单：两层卷积+池化将图像尺寸从32降到8，再通过全连接层映射到分类输出。模型参数量远小于ResNet-20，训练和推理都更快速。

在训练过程中，代理模型达到尽可能高的准确率，以便它对输入输出的映射接近目标模型。

In [12]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(proxy_model.parameters(), lr=0.001)

# 训练代理模型
epochs = 25
for epoch in range(epochs):
    running_loss = 0.0
    proxy_model.train()
    for images, labels in trainloader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = proxy_model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    # 每个epoch结束后输出一下平均损失
    print(f"Epoch {epoch+1}/{epochs}, Loss: {running_loss/len(trainloader):.4f}")

Epoch 1/25, Loss: 1.3692
Epoch 2/25, Loss: 0.9960
Epoch 3/25, Loss: 0.8355
Epoch 4/25, Loss: 0.7246
Epoch 5/25, Loss: 0.6228
Epoch 6/25, Loss: 0.5366
Epoch 7/25, Loss: 0.4521
Epoch 8/25, Loss: 0.3760
Epoch 9/25, Loss: 0.3114
Epoch 10/25, Loss: 0.2452
Epoch 11/25, Loss: 0.2007
Epoch 12/25, Loss: 0.1466
Epoch 13/25, Loss: 0.1168
Epoch 14/25, Loss: 0.0956
Epoch 15/25, Loss: 0.0881
Epoch 16/25, Loss: 0.0777
Epoch 17/25, Loss: 0.0842
Epoch 18/25, Loss: 0.0705
Epoch 19/25, Loss: 0.0619
Epoch 20/25, Loss: 0.0626
Epoch 21/25, Loss: 0.0559
Epoch 22/25, Loss: 0.0443
Epoch 23/25, Loss: 0.0473
Epoch 24/25, Loss: 0.0554
Epoch 25/25, Loss: 0.0485


接下来评估代理模型在测试集上的准确率：

In [13]:
# 评估代理模型在测试集上的准确率
proxy_model.eval()
correct = 0
total = 0
with torch.no_grad():
    for images, labels in testloader:
        images, labels = images.to(device), labels.to(device)
        outputs = proxy_model(images)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
proxy_acc = 100 * correct / total
print(f"代理模型在测试集上的准确率: {proxy_acc:.2f}%")

代理模型在测试集上的准确率: 71.03%


代理模型在测试集上达到了约71.03%的准确率。虽然低于ResNet-20的92%，但已经较好地学习了任务。这意味着代理模型的决策与目标模型有一定相似性，可用于迁移攻击。

## 5. 黑盒攻击

有了代理模型，选择**快速梯度符号法（FGSM）**作为对代理模型的攻击方法。

选择ε=0.03，扰动幅度约为0.03，在像素0~1归一化范围属于L∞范数约8/255。这个扰动强度通常能在不明显影响图像可感知性的情况下大幅降低模型准确率。

对代理模型执行FGSM攻击，并测试这些对抗样本在目标模型上的表现：

In [14]:
epsilon = 0.03

def fgsm_attack(image, epsilon, data_grad):
    # 获取梯度符号并生成扰动图像
    sign_data_grad = data_grad.sign()
    perturbed_image = image + epsilon * sign_data_grad
    # 将像素裁剪到[0,1]
    perturbed_image = torch.clamp(perturbed_image, 0, 1)
    return perturbed_image

# 在代理模型上生成对抗样本并测试在目标模型上的准确率
proxy_model.eval()
model.eval()
correct_blackbox = 0
total_blackbox = 0

for images, labels in testloader:
    images, labels = images.to(device), labels.to(device)
    images.requires_grad = True
    # 代理模型预测
    outputs = proxy_model(images)
    _, pred = torch.max(outputs, 1)
    loss = criterion(outputs, labels)
    # 计算梯度
    proxy_model.zero_grad()
    loss.backward()
    data_grad = images.grad.data
    # 生成对抗样本
    perturbed_images = fgsm_attack(images, epsilon, data_grad)
    # 用目标模型测试对抗样本
    outputs_target = model(perturbed_images)
    _, target_preds = torch.max(outputs_target, 1)
    # 统计目标模型在对抗样本上的准确数
    total_blackbox += labels.size(0)
    correct_blackbox += (target_preds == labels).sum().item()

blackbox_acc = 100 * correct_blackbox / total_blackbox
print(f"黑盒攻击下目标模型准确率: {blackbox_acc:.2f}%")

黑盒攻击下目标模型准确率: 56.22%


这表明通过代理模型产生的对抗样本在**未知的目标模型上**也造成了一定效果：ResNet-20模型的准确率从原来的93.89%降低到了约**56.22%**。这个降幅虽然不如白盒攻击剧烈，但依然显著，证明了对抗扰动的**迁移性**。

## 6. 实验结果对比与分析

将实验关键结果汇总如下表，以方便比较原始模型、白盒攻击以及黑盒攻击情况下的性能差异：

| 模型预测情况        | **整体准确率** | **各类别平均准确率** (10类) |
| :----------------- | :------------: | :------------------------: |
| **无攻击 (原始)**   | 93.89%        | 约91%（85%～97%之间）      |
| **白盒攻击 (DeepFool)** | **9.44%**    | 约10% |
| **黑盒攻击 (FGSM)**    | 56.22%       | 56%左右 |


实验结论：

1. 白盒攻击效果极端显著。攻击者利用模型完整信息，找到最小扰动即可改变模型决策。DeepFool攻击后模型对绝大多数样本都分类错误，各类别准确率大幅度下降。

2. 黑盒攻击有效，但效果有限。尽管攻击者不了解目标模型内部细节，仅凭代理模型生成对抗样本，仍然使目标模型准确率下降约40个百分点。这证明对抗扰动在不同模型间具有迁移性。不过，相比白盒，黑盒攻击的成功率明显较低。

综上，实验验证了对抗样本对深度模型的严重威胁。在白盒场景下，模型难以抵抗精心优化的扰动；在黑盒场景下，即便缺乏模型细节信息，攻击者仍可通过代理模型等方式发动有效攻击。